<a href="https://colab.research.google.com/github/JCoji/DataScience/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSI 4142 Assignment 1


Lucas Gavura, 300310069

Jonathan Cojita, student no

Febuary 3, 2026

# Shared code and dependency imports


In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import kagglehub

# Dataset 2: Adult Income
**Author:** SERPIL

**Purpose:** No specific description from author, but uses of this dataset display focus on showing correlation between demographics and income as well as making ML models to predict income based on those demographics.

**Shape:** 15 columns, 48842 rows .csv file. 5.33 MB file size

**Features (First 10):**
1.   Age (numerical): Age of worker
2.   Work class (catagorical): sector of their work (public, private, etc.)
3. fnlwgt (numerical): Final Weight, census information to show how many people this indivual 'represents'
4. education (catagorical): Represents highest level of individual's education
5. education-num (numerical): Numerical mapping of indiviudal's education level from 1(Pre-school/no formal education) - 16(Doctorate)
6. marital-status (catagorical): The individual's current marriage status
7. occupation (cataorical): Broad industry that the individual works in
8. relationship (catagorical): Familial member status of individual
9. race (catagorical): Race of individual
10. Gender (catagorical): Gender of indidual (Male and )




In [ ]:
#Import dataset and preview
path = kagglehub.dataset_download(
    "serpilturanyksel/adult-income"
)

df = pd.read_csv(f"{path}/adult11.csv")
print(df.head())
print("Lenght: ", len(df))

Using Colab cache for faster access to the 'adult-income' dataset.
   age  workclass  fnlwgt     education  education-num      marital-status  \
0   25    Private  226802          11th              7       Never-married   
1   38    Private   89814       HS-grad              9  Married-civ-spouse   
2   28  Local-gov  336951    Assoc-acdm             12  Married-civ-spouse   
3   44    Private  160323  Some-college             10  Married-civ-spouse   
4   18          ?  103497  Some-college             10       Never-married   

          occupation relationship   race  gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black    Male             0             0   
1    Farming-fishing      Husband  White    Male             0             0   
2    Protective-serv      Husband  White    Male             0             0   
3  Machine-op-inspct      Husband  Black    Male          7688             0   
4                  ?    Own-child  White  Female             0  